# COMP 484 - Practical Assignment 6

### Ramraj Chimouriya
### CE IV / I


## Book - Learning Machine learning in 6 steps

## Chapter 6 - Deep and Reinforcement learning

___

### Recurrent Neural Network (RNN)

RNN has a feedback loop, which means it feeds previous time steps into
the current step. This type of architecture generates sequences to simulate the situation
and create synthetic data. This makes it the ideal modeling choice to work on sequence
data such as speech text mining, image captioning, time series prediction, robot control,
language modeling, etc

### Long Short Term Memory (LSTM)

LSTM is an implementation of improved RNN architecture to address the issues of
general RNN, and it enables long-range dependencies.

In [1]:
import numpy as np
np.random.seed(2017)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#### Load data

In [2]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Loading data...
17473536/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)


#### Model Config

In [3]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, recurrent_dropout=0.2, dropout=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Train

In [4]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
782/782 [==============================] - 134s 169ms/step - loss: 0.4301 - accuracy: 0.7988 - val_loss: 0.3913 - val_accuracy: 0.8252
Epoch 2/5
782/782 [==============================] - 131s 167ms/step - loss: 0.2562 - accuracy: 0.8972 - val_loss: 0.3770 - val_accuracy: 0.8349
Epoch 3/5
782/782 [==============================] - 129s 165ms/step - loss: 0.1641 - accuracy: 0.9378 - val_loss: 0.4675 - val_accuracy: 0.8274
Epoch 4/5
782/782 [==============================] - 129s 165ms/step - loss: 0.1055 - accuracy: 0.9617 - val_loss: 0.7038 - val_accuracy: 0.8092
Epoch 5/5
782/782 [==============================] - 129s 164ms/step - loss: 0.0735 - accuracy: 0.9740 - val_loss: 0.6880 - val_accuracy: 0.8135


#### Evaluate

In [5]:
train_score, train_acc = model.evaluate(X_train, y_train, batch_size=batch_size)
test_score, test_acc = model.evaluate(X_test, y_test, batch_size=batch_size)

print ('Train score:', train_score)
print ('Train accuracy:', train_acc)

print ('Test score:', test_score)
print ('Test accuracy:', test_acc)

782/782 [==============================] - 13s 16ms/step - loss: 0.6880 - accuracy: 0.8135
Train score: 0.03774108365178108
Train accuracy: 0.9886800050735474
Test score: 0.6879621744155884
Test accuracy: 0.813480019569397


### Transfer Learning

Transfer learning is an area in ML that aims to utilize the knowledge gained while
solving one problem to solve a different but related problem

In [6]:
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

#### Load data

In [7]:
batch_size = 128
nb_classes = 5
nb_epoch = 5

# input image dimensions
img_rows, img_cols = 28, 28

# number of convolutional filters to use
nb_filters = 32

# size of pooling area for max pooling
pool_size = 2

# convolution kernel size
kernel_size = 3

input_shape = (img_rows, img_cols, 1)

In [8]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
X_test_gte5 = X_test[y_test >= 5]         # np_utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

11501568/11490434 [==============================] - 0s 0us/step


#### Train model for digits 0 to 4

In [9]:
def train_model(model, train, test, nb_classes):
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    model.fit(X_train, Y_train,
              batch_size=batch_size, epochs=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [10]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(nb_filters, kernel_size,
                  padding='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Conv2D(nb_filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model, (X_train_lt5, y_train_lt5), (X_test_lt5, y_test_lt5), nb_classes)

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 33s 136ms/step - loss: 1.6099 - accuracy: 0.2167 - val_loss: 1.5800 - val_accuracy: 0.3837
Epoch 2/5
240/240 [==============================] - 33s 136ms/step - loss: 1.5695 - accuracy: 0.3137 - val_loss: 1.5342 - val_accuracy: 0.5180
Epoch 3/5
240/240 [==============================] - 34s 140ms/step - loss: 1.5276 - accuracy: 0.4016 - val_loss: 1.4855 - val_accuracy: 0.6513
Epoch 4/5
240/240 [==============================] - 33s 136ms/step - loss: 1.4798 - accuracy: 0.4884 - val_loss: 1.4303 - val_accuracy: 0.7381
Epoch 5/5
240/240 [==============================] - 33s 136ms/step - loss: 1.4254 - accuracy: 0.5615 - val_loss: 1.3661 - val_accuracy: 0.8005
Test score: 1.3661460876464844
Test accuracy: 0.8005448579788208


#### Transfer existing trained model on 0 to 4 to build model for digits 5 to 9

In [11]:
# freeze feature layers and rebuild model
for layer in feature_layers:
    layer.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model, (X_train_gte5, y_train_gte5), (X_test_gte5, y_test_gte5), nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 11s 45ms/step - loss: 1.5912 - accuracy: 0.2632 - val_loss: 1.5573 - val_accuracy: 0.3874
Epoch 2/5
230/230 [==============================] - 9s 41ms/step - loss: 1.5557 - accuracy: 0.3055 - val_loss: 1.5199 - val_accuracy: 0.4536
Epoch 3/5
230/230 [==============================] - 9s 41ms/step - loss: 1.5204 - accuracy: 0.3642 - val_loss: 1.4827 - val_accuracy: 0.5421
Epoch 4/5
230/230 [==============================] - 9s 41ms/step - loss: 1.4878 - accuracy: 0.4242 - val_loss: 1.4459 - val_accuracy: 0.6377
Epoch 5/5
230/230 [==============================] - 9s 41ms/step - loss: 1.4559 - accuracy: 0.4757 - val_loss: 1.4102 - val_accuracy: 0.7060
Test score: 1.4102023839950562
Test accuracy: 0.7060275673866272
